In [14]:
# ---------------------------------------------------------------
# Build parcel-only monthly panel  (parcels → panel DataFrame)
# ---------------------------------------------------------------
import pandas as pd, numpy as np
from pathlib import Path

ROOT      = Path().resolve().parents[0]
CLEAN_DIR = ROOT / "data" / "clean"
PARCELS   = pd.read_csv(CLEAN_DIR / "parcels_jefferson_monthly.csv")

In [15]:
PARCELS["snapshot_month"] = pd.PeriodIndex(PARCELS["snapshot_month"], freq="M")
YEAR_NOW = 2025    # for age bucket

panel = (
    PARCELS
    .groupby("snapshot_month")
    .apply(lambda g: pd.Series({
        # -------- exposures ----------
        "parcels_total"       : g["parcel_id"].nunique(),
        "sqft_total"          : g["area_a"].sum(),
        "land_value_total"    : g["market_value_land"].sum(),
        "bldg_value_total"    : g["market_value_building"].sum(),
        "total_value"         : g["aexmtot"].sum(),
        "tif_value_total"     : (g["tifmlnd"] + g["tifmbld"]).sum(),

        # -------- building stock -----
        "pct_multistory"      : (g["nostory"] >= 2).mean(),
        "pct_old_40y"         : (g["yearblt"] <= YEAR_NOW - 40).mean(),
        "pct_high_grade"      : g["grade"].isin(["A", "B", "C"]).mean(),

        # -------- land-use mix -------
        "parcels_residential" : (g["landuse"].between(500, 599)).sum(),
        "parcels_commercial"  : (g["landuse"].between(600, 699)).sum(),
        "sqft_residential"    : g.loc[g["landuse"].between(500, 599), "area_a"].sum(),
        "sqft_commercial"     : g.loc[g["landuse"].between(600, 699), "area_a"].sum(),
    }))
    .reset_index()
    .sort_values("snapshot_month")
)

# optional calendar seasonality
panel["month_idx"] = np.arange(len(panel))
panel["month_sin"] = np.sin(2*np.pi*panel["month_idx"]/12)
panel["month_cos"] = np.cos(2*np.pi*panel["month_idx"]/12)

C:\Users\wrthj\AppData\Local\Temp\ipykernel_11636\1616674875.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [19]:
# save
OUT = CLEAN_DIR / "parcel_exposure_panel_monthly.csv"
panel.to_csv(OUT, index=False)
print(f"✓ panel shape {panel.shape}  ➜  {OUT}")
panel.head()

✓ panel shape (84, 17)  ➜  C:\Repositories\jefferson-township-run-forecasting\data\clean\parcel_exposure_panel_monthly.csv


,snapshot_month,parcels_total,sqft_total,land_value_total,bldg_value_total,total_value,tif_value_total,pct_multistory,pct_old_40y,pct_high_grade,parcels_residential,parcels_commercial,sqft_residential,sqft_commercial,month_idx,month_sin,month_cos
0,2018-08,5868.0,12272060.0,21665300.0,56385900.0,78051200.0,59891800.0,0.858044,0.186264,0.427403,5359.0,119.0,12125218.0,13673.0,0,0.000000,1.000000e+00
1,2018-09,5868.0,12272060.0,21665300.0,56385900.0,78051200.0,59810900.0,0.858044,0.186264,0.427573,5359.0,119.0,12125218.0,13673.0,1,0.500000,8.660254e-01
2,2018-10,5870.0,12270803.0,21665300.0,56385900.0,78051200.0,59810900.0,0.857751,0.186201,0.427428,5359.0,119.0,12123961.0,13673.0,2,0.866025,5.000000e-01
3,2018-11,5234.0,11275417.0,10929900.0,21507900.0,32437800.0,54785500.0,0.883645,0.156859,0.453573,4903.0,98.0,11173118.0,4615.0,3,1.000000,6.123234e-17
4,2018-12,5276.0,11274421.0,11071000.0,21486100.0,32557100.0,54989500.0,0.876611,0.155610,0.450152,5003.0,99.0,11221609.0,5853.0,4,0.866025,-5.000000e-01
